<img src="./images/DLI_Header.png" style="width: 400px;">

# 測驗

恭喜你完成了今天的課程！希望你在過程中學到了一些實用技能，而且從中得到樂趣。該測試一下這些技能了。在這個測驗中，你要訓練出能夠辨識新鮮與腐壞水果的新模型。你必須讓模型的驗證準確度達到 `92%`才能通過測驗，但我們期望你可以盡力拿出更好的表現。你可以運用在前幾個練習中學到的技能。具體地說，我們建議使用一些結合遷移學習、資料增強和微調的方式。將模型訓練到在驗證資料集上有至少 92% 的準確度之後，請先儲存模型，然後再評估模型的準確度。現在就開始吧！

## 資料集

在這個練習中，你要訓練出可以辨識新鮮與腐壞水果的新模型。資料集來自 [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification)，如果你在本課程結束後有興趣展開新專案，這裡有絕佳的資源。資料集結構位於 `fruits`資料夾。其中共有六種類型的水果：新鮮蘋果、新鮮柳橙、新鮮香蕉、腐壞蘋果、腐壞柳橙和腐壞香蕉。這表示模型的輸出層必須有 6 個神經元，才能成功完成分類。你也需要運用 `categorical_crossentropy`來編寫模型，因為我們的類型超過兩種。

<img src="./images/fruits.png" style="width: 600px;">

## 載入 ImageNet 基本模型

建議你從 ImageNet 上預先訓練過的模型著手。載入具有正確權重的模型，接著設定輸入形狀，再選擇移除模型的最後一層。請記得，影像有三個維度：高度、寬度和多個色頻。由於這些圖片有色彩，因此會有紅色、綠色和藍色三個色頻。我們已經幫你填入了輸入形狀。請勿對此進行變更，否則將無法通過測驗。如果你需要設定預先訓練模型的參考資料，請參閱 [notebook 05b](05b_presidential_doggy_door.ipynb)，我們在這裡實作了遷移學習。

In [2]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58892288/58889256 [==============================] - 0s 0us/step


## 凍結基本模型

接下來，我們建議你凍結基本模型，請依照 [notebook 05b](05b_presidential_doggy_door.ipynb) 中的做法。這麼做是為了避免所有來自 ImageNet 資料集的學習在初始訓練中遭到破壞。

In [3]:
# Freeze base model
base_model.trainable = False

## 將各層新增至模型

現在將各層新增到預先訓練模型。可以將 [Notebook 05b](05b_presidential_doggy_door.ipynb) 當作操作指引。請密切注意最後的密集層，並確保這一層有正確數量的神經元，才能為不同類型的水果分類。

In [6]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [7]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## 編寫模型

現在可以用損失和指標選項來編寫模型了。請記得，我們要針對許多不同的類別進行訓練，而不是要解決二元分類問題。

In [8]:
model.compile(loss = keras.losses.BinaryCrossentropy(from_logits=True) , metrics = [keras.metrics.BinaryAccuracy()])

## 增強資料

如果你願意，可以嘗試增強資料來改善資料集。歡迎參閱 [notebook 04a](04a_asl_augmentation.ipynb) 和 [notebook 05b](05b_presidential_doggy_door.ipynb) 來瞭解增強範例。此外，我們也有提供 [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class) 的說明文件。你可以自行選擇是否要採取這個步驟，但這一步也許會有助於讓模型達到 92% 的準確度。

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)

## 載入資料集

現在該載入訓練和驗證資料集了。選擇適合的資料夾，以及適合的影像 `target_size`(必須與你建立的模型的高度和寬度輸入相符)。如果你需要參考資料，可以查看 [notebook 05b](05b_presidential_doggy_door.ipynb)。

In [10]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('fruits/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('fruits/train/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 1182 images belonging to 6 classes.


## 訓練模型

該訓練模型了！將 `train`和 `valid` 迭代器傳遞到 `fit` 函數，並且設定所需的 Epoch 數量。

In [11]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=10)

Epoch 1/10
37/36 [==============================] - 38s 1s/step - loss: 0.7239 - binary_accuracy: 0.8130 - val_loss: 0.7055 - val_binary_accuracy: 0.8505
Epoch 2/10
37/36 [==============================] - 30s 798ms/step - loss: 0.6949 - binary_accuracy: 0.8700 - val_loss: 0.6786 - val_binary_accuracy: 0.9057
Epoch 3/10
37/36 [==============================] - 30s 798ms/step - loss: 0.6727 - binary_accuracy: 0.9169 - val_loss: 0.6671 - val_binary_accuracy: 0.9271
Epoch 4/10
37/36 [==============================] - 29s 794ms/step - loss: 0.6663 - binary_accuracy: 0.9284 - val_loss: 0.6635 - val_binary_accuracy: 0.9343
Epoch 5/10
37/36 [==============================] - 30s 801ms/step - loss: 0.6632 - binary_accuracy: 0.9353 - val_loss: 0.6600 - val_binary_accuracy: 0.9411
Epoch 6/10
37/36 [==============================] - 30s 798ms/step - loss: 0.6606 - binary_accuracy: 0.9408 - val_loss: 0.6583 - val_binary_accuracy: 0.9443
Epoch 7/10
37/36 [==============================] - 30s 802ms

## 取消凍結模型以進行微調

如果驗證準確度已經達到 92%，你可以自行選擇是否要進行下個步驟。如果尚未達到，建議你以極低的學習率微調模型。

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = keras.losses.BinaryCrossentropy(from_logits=True) , metrics = [keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=15)

## 評估模型

在理想狀況下，現在你的模型驗證準確度應該會達到 92% 或以上。如果不是如此，建議你回到先前步驟執行更多 Epoch 的訓練，或是調整資料增強。

達到理想的驗證準確度後，請執行下列儲存格來評估模型。評估函數會回傳一個元組(Tuple)，其中第一個值表示損失，第二個值表示準確度。若要通過本測驗，模型的準確度值必須為 `92% or higher`。

In [12]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

37/36 [==============================] - 14s 387ms/step - loss: 0.6407 - binary_accuracy: 0.9803


[0.6406662464141846, 0.9802595376968384]

## 執行測驗

若要測驗模型，請執行以下兩個儲存格。

**注意：** `run_assessment` 會假定你的模型名稱為 `model`，以及你的驗證資料迭代器名稱為 `valid_it`。若你修改了這些變數名稱，請更新傳遞至 `run_assessment`的引數名稱。

In [13]:
from run_assessment import run_assessment

In [14]:
run_assessment(model, valid_it)

Evaluating model 5 times to obtain average accuracy...

37/36 [==============================] - 14s 390ms/step - loss: 0.6405 - binary_accuracy: 0.9822

Accuracy required to pass the assessment is 0.92 or greater.
Your average accuracy is 0.9823.

Congratulations! You passed the assessment!
See instructions below to generate a certificate.


## 產生證書

如果你通過測驗，請返回課程頁面 (如下所示)，然後按一下 [ASSESS TASK] (測驗工作) 按鈕，之後系統就會產生你的課程證書。

<img src="./images/assess_task.png" style="width: 800px;">